# **WARNING**

In order for this notebook to run, you must first run the notebook `analysis/0_analysis_skill/0_0_generate_skill_models.ipynb.ipynb` and generate the files `data_parameter/player_gaussin_fit/grid_full/e{x}_player{y}_gaussin_prob_grid.pkl` for all epsilon x and player y that you want to consider when using this notebook. These files contain the transition probability distributions that were calculated by integrating the bivariate Gaussian distributions for a given player-epsilon combination. 

# 1.0 Notebook Parameters

Please specify the following variables:
1. `project_directory`: your project directory path
2. `playerID_list`: the list of players (i.e. skill models) you want to consider in your analysis; usually use one base model and scale it with different epsilons
3. `epsilon_list`: the list of execution error multiplers (epsilon) that you want to consider in your analysis.

In [1]:

# Change this to the project directory path for your computer  
# project_directory = '/Users/rachael/Desktop/darts-thesis'
# project_directory = '/Users/Craig/PycharmProjects/darts-thesis'
project_directory = '/Users/rachael/Desktop/test/darts-thesis'

# Set the ID(s) of the player skill models you want to use 
    # 10 - symmetric DB skill model that we were using up until now 
    # "AVG" - "average pro" skill model generated by H & W 
    
playerID_list = ['AVG']
#epsilon_list = [1,2,3,4,5,6,7,8,9]
epsilon_list = [1,2,3,4,5,6,7,8,9,10,12,14,16,18,20,32,64,128,25,50,100]

# 2.0 Notebook Setup

Import libraries and run prerequisites.

In [2]:
# Import and configure modules from this project

import os

os.chdir(project_directory)

import function_helpers_singleplayer as h 
import function_init_board as fb 

data_parameter_dir = fb.data_parameter_dir
result_dir = './result/singlegame_results'  

/Users/rachael/miniconda3/envs/darts/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Import and configure external modules 

import pandas as pd

import numpy as np
np.set_printoptions(precision=4)
np.set_printoptions(linewidth=300)
np.set_printoptions(threshold=300)

import torch
torch.set_printoptions(precision=4)
torch.set_printoptions(linewidth=300)
torch.set_printoptions(threshold=300)

# This makes the code output easier to read
# Can comment out if you want to see warnings 
import warnings
warnings.filterwarnings("ignore")

# 3.0 Single Player Sandbox

Can alter the parameters and examine results for a single player here.

In [4]:
player_num = "AVG"
epsilon = 1
name_pa = 'player{}'.format(player_num)

[aiming_grid, prob_grid_normalscore_nt, prob_grid_singlescore_nt, prob_grid_doublescore_nt, prob_grid_triplescore_nt, prob_grid_bullscore_nt] = h.load_aiming_grid(name_pa, epsilon=epsilon, data_parameter_dir=data_parameter_dir, grid_version='custom_no_tokens')
[aiming_grid, prob_grid_normalscore_t, prob_grid_singlescore_t, prob_grid_doublescore_t, prob_grid_triplescore_t, prob_grid_bullscore_t] = h.load_aiming_grid('t', data_parameter_dir=data_parameter_dir, grid_version='custom_tokens')

prob_grid_doublescore_dic = None
prob_grid_doublescore_dic_t = None

[optimal_value, optimal_action_index] = h.solve_dp_noturn_tokens(aiming_grid, prob_grid_normalscore_nt, prob_grid_normalscore_t, tokens = 9, prob_grid_doublescore=prob_grid_doublescore_nt, prob_grid_bullscore=prob_grid_bullscore_nt, prob_grid_doublescore_dic=None, prob_grid_doublescore_t=prob_grid_doublescore_t, prob_grid_bullscore_t=prob_grid_bullscore_t, prob_grid_doublescore_dic_t=None)

print('------------------------------------------------------------------')
print('Optimal Values: ')
print(optimal_value)
print('------------------------------------------------------------------')
print('Optimal Policy}: ')
print(optimal_action_index)

load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_no_tokens/playerAVG_e1_gaussin_prob_grid_custom_no_tokens.pkl
load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_tokens/t_gaussin_prob_grid_custom_tokens.pkl
------------------------------------------------------------------
Optimal Values: 
[[ 0.      0.      2.3502 ... 14.7497 14.772  14.8449]
 [ 0.      0.      1.     ... 12.9194 12.8639 12.9152]
 [ 0.      0.      1.     ... 12.3866 12.325  12.3265]
 ...
 [ 0.      0.      1.     ...  9.9608  9.9378  9.9389]
 [ 0.      0.      1.     ...  9.4919  9.4883  9.4921]
 [ 0.      0.      1.     ...  9.      9.      9.    ]]
------------------------------------------------------------------
Optimal Policy}: 
[[ -1  -1  55 ...  11  11  11]
 [ -1  -1 752 ...  11  11  11]
 [ -1  -1 752 ...  11  11  11]
 ...
 [ -1  -1 752 ...  11  11  11]
 [ -1  -1 752 ...  11  11  11]
 [ -1  -1 752 ... 750 750 750]]


# 4.0 Generate Dataset


In [5]:
# Helper function to deconstruct the solved MDPs and reformat as a dataframe 
def create_result_dataframes(player, optimal_value,optimal_action_index):
    
    col_names = ['player','epsilon','tokens','score','optimal_value','optimal_policy']
    df_list = []
    
    for t in range(optimal_value.shape[0]):
        for s in range(optimal_value.shape[1]):
            df_list.append([player,epsilon,t,s,optimal_value[t][s],optimal_action_index[t][s]])

    df = pd.DataFrame(df_list,columns=col_names)
    
    return df 


In [6]:
col_names = ['player','epsilon','tokens','score','optimal_value','optimal_policy']
df = pd.DataFrame(columns=col_names)

for player_num in playerID_list:

    name_pa = 'player{}'.format(player_num)

    [aiming_grid, prob_grid_normalscore_t, prob_grid_singlescore_t, prob_grid_doublescore_t, prob_grid_triplescore_t, prob_grid_bullscore_t] = h.load_aiming_grid('t', data_parameter_dir=data_parameter_dir, grid_version='custom_tokens')
    prob_grid_doublescore_dic = None
    prob_grid_doublescore_dic_t = None

    for epsilon in epsilon_list:
        [aiming_grid, prob_grid_normalscore_nt, prob_grid_singlescore_nt, prob_grid_doublescore_nt, prob_grid_triplescore_nt, prob_grid_bullscore_nt] = h.load_aiming_grid(name_pa, epsilon=epsilon, data_parameter_dir=data_parameter_dir, grid_version='custom_no_tokens')
        [optimal_value, optimal_action_index] = h.solve_dp_noturn_tokens(aiming_grid, prob_grid_normalscore_nt, prob_grid_normalscore_t, tokens = 9, prob_grid_doublescore=prob_grid_doublescore_nt, prob_grid_bullscore=prob_grid_bullscore_nt, prob_grid_doublescore_dic=None, prob_grid_doublescore_t=prob_grid_doublescore_t, prob_grid_bullscore_t=prob_grid_bullscore_t, prob_grid_doublescore_dic_t=None)
        df = pd.concat([df,create_result_dataframes(player_num,optimal_value,optimal_action_index)])
        df = df.reset_index().drop(['index'],axis=1)


load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_tokens/t_gaussin_prob_grid_custom_tokens.pkl
load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_no_tokens/playerAVG_e1_gaussin_prob_grid_custom_no_tokens.pkl
load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_no_tokens/playerAVG_e2_gaussin_prob_grid_custom_no_tokens.pkl
load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_no_tokens/playerAVG_e3_gaussin_prob_grid_custom_no_tokens.pkl
load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_no_tokens/playerAVG_e4_gaussin_prob_grid_custom_no_tokens.pkl
load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_no_tokens/playerAVG_e5_gaussin_prob_grid_custom_no_tokens.pkl
load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_no_tokens/playerAVG_e6_gaussin_prob_grid_custom_no_tokens.pkl
load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_no_tokens/playerAVG_e7_gaussin_prob_grid_custom_no_tokens.pkl
l

In [7]:
# Save dataset 
result_path = result_dir + '/playerAVG_noturn_results.csv'
if not os.path.isdir(result_dir):
    os.makedirs(result_dir)
df.to_csv(result_path)

In [8]:
result_dir

'./result/singlegame_results'